In [1]:
import sys
import os

BASE_DIR = os.getcwd().rsplit("/",1)[0]
sys.path.append(BASE_DIR)
print("adding Base Dir Path", BASE_DIR)


from pytorch_lightning import Trainer
from monoscene.models.monoscene import MonoScene
from monoscene.data.NYU.nyu_dm import NYUDataModule
from monoscene.data.semantic_kitti.kitti_dm import KittiDataModule
from monoscene.data.kitti_360.kitti_360_dm import Kitti360DataModule
import hydra
from omegaconf import DictConfig
import torch
import numpy as np
import os
from hydra.utils import get_original_cwd
from tqdm import tqdm
import pickle




adding Base Dir Path /monoscene/MonoScene


# Config

In [2]:
import yaml
from types import SimpleNamespace

with open("../monoscene/config/monoscene.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)
        config = SimpleNamespace(**config)
    except yaml.YAMLError as exc:
        print(exc)

# Data Loader

In [3]:
project_scale = 1
feature = 200
full_scene_size = (60, 36, 60)
data_module = NYUDataModule(
    root=config.NYU_root,
    preprocess_root=config.NYU_preprocess_root,
    n_relations=config.n_relations,
    frustum_size=config.frustum_size,
    batch_size=1,#int(config.batch_size / config.n_gpus),
    num_workers=1#int(config.num_workers_per_gpu * config.n_gpus),
)
data_module.setup()
data_loader = data_module.val_dataloader()
data_loader = data_module.test_dataloader() # use this if you want to infer on test set

# Loading Model

In [4]:
model_path = "/monoscene/MonoScene/trained_models/monoscene_nyu.ckpt"
model = MonoScene.load_from_checkpoint(
        model_path,
        feature=feature,
        project_scale=project_scale,
        fp_loss=config.fp_loss,
        full_scene_size=full_scene_size,
    )
model.cuda()
model.eval()

n_relations 4
Loading base model ()...

Using cache found in /root/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


Done.
Removing last two layers (global_pool & classifier).
Building Encoder-Decoder model..Done.


MonoScene(
  (projects): ModuleDict(
    (1): FLoSP()
    (2): FLoSP()
    (4): FLoSP()
    (8): FLoSP()
  )
  (net_3d_decoder): UNet3D(
    (process_1_4): Sequential(
      (0): Process(
        (main): Sequential(
          (0): Bottleneck3D(
            (conv1): Conv3d(200, 50, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
            (bn1): BatchNorm3d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv2): Conv3d(50, 50, kernel_size=(1, 1, 3), stride=(1, 1, 1), padding=(0, 0, 1), bias=False)
            (bn2): BatchNorm3d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv3): Conv3d(50, 50, kernel_size=(1, 3, 1), stride=(1, 1, 1), padding=(0, 1, 0), bias=False)
            (bn3): BatchNorm3d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv4): Conv3d(50, 50, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
            (bn4): BatchNorm3d(50, eps=1e-05

# Inference

In [5]:
output_path = '/monoscene/Monoscene/outputs/'
with torch.no_grad():
    for batch in tqdm(data_loader):
        batch["img"] = batch["img"].cuda()
        pred = model(batch)
        y_pred = torch.softmax(pred["ssc_logit"], dim=1).detach().cpu().numpy()
        y_pred = np.argmax(y_pred, axis=1)
        for i in range(config.batch_size):
            out_dict = {"y_pred": y_pred[i].astype(np.uint16)}
            if "target" in batch:
                out_dict["target"] = (
                    batch["target"][i].detach().cpu().numpy().astype(np.uint16)
                )

            if config.dataset == "NYU":
                write_path = output_path
                filepath = os.path.join(write_path, batch["name"][i] + ".pkl")
                out_dict["cam_pose"] = batch["cam_pose"][i].detach().cpu().numpy()
                out_dict["vox_origin"] = (
                    batch["vox_origin"][i].detach().cpu().numpy()
                )
            else:
                write_path = os.path.join(output_path, batch["sequence"][i])
                filepath = os.path.join(write_path, batch["frame_id"][i] + ".pkl")
                out_dict["fov_mask_1"] = (
                    batch["fov_mask_1"][i].detach().cpu().numpy()
                )
                out_dict["cam_k"] = batch["cam_k"][i].detach().cpu().numpy()
                out_dict["T_velo_2_cam"] = (
                    batch["T_velo_2_cam"][i].detach().cpu().numpy()
                )

            os.makedirs(write_path, exist_ok=True)
            with open(filepath, "wb") as handle:
                pickle.dump(out_dict, handle)
                print("wrote to", filepath)

# Debug

In [6]:

for batch in tqdm(data_loader):
    batch["img"] = batch["img"].cuda()
    print(batch)
    break


  0%|          | 0/654 [00:00<?, ?it/s]


RuntimeError: DataLoader worker (pid(s) 1003) exited unexpectedly